# 跑步天气
## 文档
### 需求
1. 分时取温度，并对温度、天气进行可视化，并给出相应跑步穿衣指南
（更像一个前端项目，基本没有数据分析）
2. 每日/分时天气预测

### 1 数据来源：
1. 当日天气：https://www.tianqi.com/beijing/
2. 历史天气：http://www.tianqihoubao.com/lishi/beijing.html
3. 历史pm值：http://www.tianqihoubao.com/aqi/beijing.html
### 2 爬取数据
爬取工具：request
### 3 数据库
mysql
### 4 线下预测模型
### 5 预测及可视化

## 需求数据
序号|编码|列名
:--:|:--:|:--:|:--:
1|id|主键|int
2|name|地区|str
3|time|时间|datetime
4|weather|天气|int
5|max_temp|最高气温|int
6|min_temp|最低气温|int
7|cur_temp|当前气温|int
8|humidity|当前湿度|int
9|wind_dir|风向|str
10|wind_power|风力|int
11|rays|紫外线|int
12|pm|pm值|int

# 数据库
```mysql
create table weather_data.weather_data2 (
  id int not null auto_increment comment '主键',
  name varchar(80) COLLATE utf8_bin null default '' comment '地区',
  time datetime not null default CURRENT_TIMESTAMP comment '时间',
  weather varchar(50) COLLATE utf8_bin default '' comment '天气',
  max_temp int not null default 100 comment '最高温度',
  min_temp int not null default 100 comment '最低温度',
  cur_temp int not null default 100 comment '当前温度',
  humidity int not null default 100 comment '当前湿度',
  wind_dir varchar(50) not null  comment '风向',
  wind_power int not null default -1 comment '风力',
  rays int not null default -1 comment '紫外线',
  pm int not null default -1 comment 'PM值',
  primary key (id)
) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin comment '天气数据';
```

## 爬取数据
2.1 爬虫：request

2.2 解析：parse

In [1]:
import requests
import bs4
import pymysql

In [2]:
res = requests.get('https://www.tianqi.com/beijing/')
try:
    res.raise_for_status()
except Exception as exc:
    print(exc)

In [3]:
#数据采集
weather_soup = bs4.BeautifulSoup(res.text, 'lxml')
soup = weather_soup.select('.weather_info')

name = soup[0].select('.name h2')[0].contents[0]
time = soup[0].select('.week')[0].contents[0]
climate = soup[0].select('span b')[0].contents[0]
temperature = soup[0].select('span')[0].contents[1]
now_temp = soup[0].select('.now b')[0].contents[0]
humidity = soup[0].select('.shidu b')[0].contents[0]
wind = soup[0].select('.shidu b')[1].contents[0]
rays = soup[0].select('.shidu b')[2].contents[0]
pm = soup[0].select('.kongqi h6')[0].contents[0]



#181222天气
~~~html
[<dl class="weather_info">
<dt><img alt="北京天气预报" src="http://content.pic.tianqistatic.com/content/20171123/7a229585d3cb9763f27f4a37611c7083.gif"/></dt>
<dd class="name"><h2>北京</h2><i><a href="/chinacity.html">[切换城市]</a></i></dd>
<dd class="week">2018年12月22日　星期六　戊戌年冬月十六 </dd>
<dd class="weather">
<i><img src="//static.tianqistatic.com/static/wap2018/ico1/b1.png"/></i>
<p class="now"><b>7</b><i>℃</i></p>
<span><b>多云</b>-2 ~ 8℃</span>
</dd>
<dd class="shidu"><b>湿度：19%</b><b>风向：西北风 2级</b><b>紫外线：最弱</b></dd>
<dd class="kongqi"><h5 style="background-color:#79b800;">空气质量：优</h5><h6>PM: 15</h6><span>日出: 07:32<br/>日落: 16:52</span></dd>
<dl>
</dl></dl>]
~~~

In [4]:
### 优化函数
#def cut_str(string,intab,start=0,end=-1):
wind

'风向：西北风 3级'

In [98]:
print(soup)
str(name)

[<dl class="weather_info">
<dt><img alt="北京天气预报" src="http://content.pic.tianqistatic.com/content/20171123/7a229585d3cb9763f27f4a37611c7083.gif"/></dt>
<dd class="name"><h2>北京</h2><i><a href="/chinacity.html">[切换城市]</a></i></dd>
<dd class="week">2018年12月22日　星期六　戊戌年冬月十六 </dd>
<dd class="weather">
<i><img src="//static.tianqistatic.com/static/wap2018/ico1/b1.png"/></i>
<p class="now"><b>7</b><i>℃</i></p>
<span><b>多云</b>-2 ~ 8℃</span>
</dd>
<dd class="shidu"><b>湿度：19%</b><b>风向：西北风 2级</b><b>紫外线：最弱</b></dd>
<dd class="kongqi"><h5 style="background-color:#79b800;">空气质量：优</h5><h6>PM: 15</h6><span>日出: 07:32<br/>日落: 16:52</span></dd>
<dl>
</dl></dl>]


'北京'

In [4]:
#数据转换及储存
time = time[0:time.find('日')]
min_temp = temperature[0:temperature.find('~')]
max_temp = temperature[temperature.find('~')+1:temperature.find('℃')]
cur_temp = now_temp[:]
humidity = humidity[humidity.find('：')+1:-1]
wind_dir = wind[wind.find('：')+1:wind.rfind('风')]
wind_power = wind[wind.rfind('风')+1:-1]
rays = rays[rays.find('：')+1:]
pm = pm[pm.find(':')+1:]

weather_dict = {}
rays_dict = {'最弱':0,'弱':1,'中等':2,'强':3,'最强':4}
weather_dict['name'] = str(name)
weather_dict['time'] = time.translate(str.maketrans('年月','--'))
weather_dict['weather'] = str(climate)
weather_dict['max_temp'] = int(max_temp)
weather_dict['min_temp'] = int(min_temp)
weather_dict['cur_temp'] = int(now_temp)
weather_dict['humidity'] = int(humidity)
weather_dict['wind_dir'] = str(wind_dir)
weather_dict['wind_power'] = int(wind_power)
weather_dict['rays'] = rays_dict[rays]
weather_dict['pm'] = int(pm)
weather_dict

{'name': '北京',
 'time': '2019-01-06',
 'weather': '晴',
 'max_temp': 3,
 'min_temp': -7,
 'cur_temp': -5,
 'humidity': 37,
 'wind_dir': '东',
 'wind_power': 1,
 'rays': 3,
 'pm': 29}

In [5]:
#导入mysql
import pymysql
db = pymysql.connect(host="localhost",
                     user="root",
                     password="q1889233",
                     db='weather_data',
                     charset='utf8mb4',
                     cursorclass=pymysql.cursors.DictCursor )

In [6]:
####
for i in weather_dict:
    print(i,weather_dict[i],type(weather_dict[i]))



name 北京 <class 'str'>
time 2019-01-06 <class 'str'>
weather 晴 <class 'str'>
max_temp 3 <class 'int'>
min_temp -7 <class 'int'>
cur_temp -5 <class 'int'>
humidity 37 <class 'int'>
wind_dir 东 <class 'str'>
wind_power 1 <class 'int'>
rays 3 <class 'int'>
pm 29 <class 'int'>


In [7]:
#插入数据
#(weather['name'],weather['time'],weather['climate'],weather['max_temp'],weather['min_temp'],
#           weather['humidity'],weather['wind'],weather['rays'],weather['pm'])
with db.cursor() as cursor:

    sql='''INSERT INTO `weather_data2`(`name`,`time`,`weather`,`max_temp`,`min_temp`,`cur_temp`,`humidity`,`wind_dir`,`wind_power`,`rays`,`pm`) 
           VALUES (%s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s)'''
    cursor.execute(sql,(str(weather_dict['name']),weather_dict['time'],str(weather_dict['weather']),weather_dict['max_temp'],weather_dict['min_temp'],
           weather_dict['cur_temp'],weather_dict['humidity'],weather_dict['wind_dir'],weather_dict['wind_power'],weather_dict['rays'],weather_dict['pm']))

db.commit()
    

In [8]:
#从db中取出数据
#try:
with db.cursor() as cursor:
    sql = "SELECT * FROM `weather_data2` ORDER BY `time` DESC LIMIT 3"
    cursor.execute(sql)
    #只取出一条结果
    result=cursor.fetchall()
    print(result)

#最后别忘了关闭连接
#finally:
#   db.close()

[{'id': 10, 'name': '北京', 'time': datetime.datetime(2019, 1, 6, 0, 0), 'weather': '晴', 'max_temp': 3, 'min_temp': -7, 'cur_temp': -5, 'humidity': 37, 'wind_dir': '东', 'wind_power': 1, 'rays': 3, 'pm': 29}, {'id': 9, 'name': '北京', 'time': datetime.datetime(2019, 1, 5, 0, 0), 'weather': '多云', 'max_temp': 0, 'min_temp': -8, 'cur_temp': -2, 'humidity': 18, 'wind_dir': '东', 'wind_power': 1, 'rays': 0, 'pm': 18}, {'id': 8, 'name': '北京', 'time': datetime.datetime(2019, 1, 4, 0, 0), 'weather': '晴', 'max_temp': 2, 'min_temp': -7, 'cur_temp': -1, 'humidity': 18, 'wind_dir': '西北', 'wind_power': 3, 'rays': 3, 'pm': 14}]


In [10]:
###
import pandas as pd
df_ofa = pd.DataFrame(result)
df_ofa

,cur_temp,humidity,id,max_temp,min_temp,name,pm,rays,time,weather,wind_dir,wind_power
0,-5,37,10,3,-7,北京,29,3,2019-01-06,晴,东,1
1,-2,18,9,0,-8,北京,18,0,2019-01-05,多云,东,1
2,-1,18,8,2,-7,北京,14,3,2019-01-04,晴,西北,3


In [8]:
###
df_ora

NameError: name 'df_ora' is not defined

In [8]:
result

{'id': 1,
 'name': '北京',
 'time': datetime.datetime(2018, 12, 23, 0, 0),
 'weather': '晴',
 'max_temp': 0,
 'min_temp': -5,
 'cur_temp': -1,
 'humidity': 12,
 'wind_dir': '西南',
 'wind_power': 2,
 'rays': 3,
 'pm': 9}

In [52]:
#weather_soup.find('weatherbox').contents
#name = weather_soup.select('.weatherbox .weather_info .name h2')


In [51]:
soup[0].select('.kongqi h6')[0].contents[0]

'PM: 23'

In [9]:
len(res.text)
print(res.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<title>【北京天气预报】北京天气预报一周_北京天气预报10天、15天查询—天气网</title>
<meta content="★天气网（www.tianqi.com）★北京天气预报提供北京今日天气、明天天气以及北京未来一周的天气预报,可以实时查看北京天气预报一周、10天、15天的天气情况。旅游出行,从天气网开始！" name="description">
<meta name="keywords" content="北京天气预报查询,北京天气预报一周,北京天气预报10天,北京天气预报查询15天, 北京未来一周的天气预报">
<meta http-equiv='mobile-agent' content='format=xhtml;url=https://m.tianqi.com/beijing/'>
	<meta name="referrer" content="unsafe-url" />
<link rel='canonical' href='https://www.tianqi.com/beijing/' >
	<!--<link href="//static.tianqistatic.com/static/css/new_tianqi.css" rel="stylesheet" type="text/css">-->
	<link href="//static.tianqistatic.com/static/tianqi2018/css/news.css" rel="stylesheet" type="text/css">
	<link href="//static.tianqistatic.com/static/css/pub.css" rel="stylesheet" ty